# Bitmoji Facial Segmentation Model

[Reference 1](https://towardsdatascience.com/building-a-custom-semantic-segmentation-model-abb0843ac12d)

[Reference 1 Git](https://github.com/sam-watts/futoshiki-solver)

[Reference 2](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [1]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from tqdm.notebook import tqdm
import segmentation_models_pytorch as smp
import seaborn as sns
import pandas as pd
import numpy as np
import torch

In [2]:
from utils import Rescale,RandomCrop,ToTensor,FaceLandmarksDataset

transformed_dataset = FaceLandmarksDataset(csv_file='../data/bitmoji_annotation.csv',
                                           root_dir='../data/training/',
                                           transform=transforms.Compose([Rescale((400,600)),
                                                                         RandomCrop((320, 532)),
                                                                         ToTensor()]))


dataloader = DataLoader(transformed_dataset, batch_size=1,
                        shuffle=True)


## Image augmentation

To get the most out of our labelled data we'll use the library [albumentations](https://albumentations.ai/) to add noise to the dataset

In [3]:
import albumentations as albu

def get_training_augmentation():
    train_transform = [
        albu.HorizontalFlip(p=0.5),
        albu.ShiftScaleRotate(scale_limit=0.2, rotate_limit=20, shift_limit=0.2, p=0.8, border_mode=0),
        albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),
        albu.IAAAdditiveGaussianNoise(p=0.1),
        albu.IAAPerspective(p=0.5),
        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        )
    ]
    
    return albu.Compose(train_transform)

In [4]:
import segmentation_models_pytorch as smp
import torch

# Load basemodel
ENCODER = 'efficientnet-b3'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['face']
ACTIVATION = 'sigmoid'
DEVICE = torch.device('cpu')

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
).to(DEVICE)